In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/home/sansforensics/Desktop/Blucoat_Pandas/Demo_log_004.log',engine='python', sep=' ',error_bad_lines=False)
df

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
# 1) Uncategorized sites -> Depending on proxy uncategorized site can have diff name

In [ ]:
df[df['sc-filter-category'] == "unavailable"]['cs-host'].unique()

In [ ]:
df[df['sc-filter-category'] == "unavailable"]['c-ip'].unique()

In [ ]:
# 2) Hits toward malicious sites and count ( categories ) 

In [ ]:
df['sc-filter-category'].unique()

In [ ]:
df [ df['sc-filter-category'].str.contains('mal|Mal|bot|spy',case=False) == True]

In [ ]:
df [ df['sc-filter-category'].str.contains('mal|Mal|bot|spy',case=False) == True]['cs-host'].values

In [ ]:
df [ df['sc-filter-category'].str.contains('mal|Mal|bot|spy',case=False) == True]['cs-host'].unique()

In [ ]:
df [ df['sc-filter-category'].str.contains('mal|Mal|bot|spy',case=False) == True]['cs-host'].value_counts()

In [ ]:
# 3) IP based communication. Take this data and check reputation of ip's for further analysis

In [ ]:
df [ df['cs-host'].str.contains(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}') == True ]['cs-host'].unique()

In [ ]:
df [df['cs-host'].str.contains(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}') == True ][['cs-host','sc-status']]


In [ ]:
# 4)  HTTP POST / PUT method usage......

In [ ]:
df [(df['cs-method'] == 'POST') | (df['cs-method'] == 'PUT')][['c-ip','cs-host','cs-bytes']]

In [ ]:
# 5) sites where max data is uploaded

In [ ]:
temp = df.groupby('cs-host').sum().sort_values('cs-bytes',ascending=False) 
temp [ test > 1000000 ]['cs-bytes'].sort_values(ascending=False)

In [ ]:
# 6) Playing with Content Type

In [ ]:
# List of content 
df['rs(Content-Type)'].unique()

In [ ]:
# POST method uses application/x-www-urlform-encoded to upload data
df [ ( df['rs(Content-Type)'] == 'application/x-www-urlform-encoded' ) == True ]['cs-host'].unique()

In [ ]:
# when executables are downloaded this content type is seen
df [ (df['rs(Content-Type)'] == 'application/octate-stream') == True ]

In [ ]:
#images with malicious content
df[ (df['cs-uri-path'].str.contains('png|PNG') ) & ( df['rs(Content-Type)'] == 'application/octate-stream') ]['rs(Content-Type)']

In [ ]:
# 7)  User Agent string checks

In [ ]:
# hunitng for powershell | python | curl | office  User agent string
df [df['cs(User-Agent)'].str.contains('pow|office|python|curl', case= False) == True]['cs(User-Agent)'].unique()

In [ ]:
# less occuring UA
temp = df['cs(User-Agent)'].value_counts() < 5
temp.tail(10).index

In [ ]:
# print UA with values
temp = df['cs(User-Agent)'].value_counts()[df['cs(User-Agent)'].value_counts() < 5 ]
temp

In [ ]:
# 8)  Quick check with whois data - To check recently registered domain

In [ ]:
import whois
temp = df [ df ['cs(User-Agent)'] == 'Windows-Update-Agent']['cs-host'].unique()

for i in temp:
    c_date =  str ( whois.whois(i).creation_date )
    print(i + "\t" + c_date)

In [ ]:
# 9) Adding column based on certain criteria - In this data set, there were no Usernames so I just added with below conditions.

In [ ]:
import numpy as np
conditions = [
    (df['sc-bytes'] <= 500),
    (df['sc-bytes'] > 500) & (df['sc-bytes'] <= 1000),
    (df['sc-bytes'] > 1000) & (df['sc-bytes'] <= 1500),
    (df['sc-bytes'] > 1500)
    ]

# create a list of the values we want to assign for each condition
values = ['sanket', 'suraj', 'parth', 'jay']

# create a new column and use np.select to assign values to it using our lists as arguments
df['new_cs_user'] = np.select(conditions, values)

# display updated DataFrame
df.head()

In [ ]:
# 10 )  User uploading data

In [ ]:
# adding  new column with data in Kb
df['cs-bytes-in-Kb'] = df['cs-bytes'].apply(lambda x : x/1000)
df['cs-bytes-in-Kb'].sort_values(ascending=False)

# POST method check
df.groupby('cs-host').sum()['cs-bytes-in-Kb']
temp = df[df['cs-method'] == "POST"]
for u in temp.groupby('new_cs_user').groups.keys(): # to iterate users
    site = temp [ temp['new_cs_user'] == u]         # create dataframe based on user   
    site_grp = site.groupby('cs-host').sum()['cs-bytes-in-Kb']  # groupby website name and sum the upload data
    print("user uploading is ", u)
    print ( site_grp [ site_grp > 1000 ] ) # show more than 1000 Kb data

In [ ]:
#Thank You